In [ ]:
import torch
if torch.cuda.is_available(): 
    print(f"{torch.cuda.device_count()} GPUs available.")
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import networkx as nx
from glob import glob
import numpy as np
import pandas as pd
from livelossplot import PlotLosses
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.rc('text', usetex=False)
plt.rc('font', family='serif')

# ### Set seeds for reproducibility
#seed = 15
#torch.manual_seed(seed)
#torch.cuda.manual_seed(seed)
#np.random.seed(seed)
#import random
#random.seed(seed)

## Test workflow on a single stream

In [ ]:
def create_subset(df): 
    # Fit a line to the subset using linear regression
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    stream_ϕ = df[df.stream]['ϕ'].values.reshape(-1, 1)
    stream_λ = df[df.stream]['λ'].values
    model.fit(stream_ϕ, stream_λ)
    
    # Get the slope and intercept of the fitted line
    slope = model.coef_[0]
    intercept = model.intercept_
    
    # Function to calculate perpendicular distance from a point to a line
    def distance_to_line(x, y, slope, intercept):
        return abs(slope * x - y + intercept) / np.sqrt(slope**2 + 1)
    
    # Define the distance threshold
    max_star_distance = np.max(distance_to_line(df[df.stream]['ϕ'].values, df[df.stream]['λ'].values, slope, intercept))
    radius = np.max([2.0, max_star_distance])
    
    # Calculate distances of all points to the fitted line
    distances = distance_to_line(df['ϕ'].values, df['λ'].values, slope, intercept)
    
    # Select points within the distance threshold
    selected_points = df[distances <= radius]

    return selected_points 

In [ ]:
mock_streams = sorted(glob("/global/cfs/cdirs/m3246/mpettee/misc/GaiaCWoLa/gaia_data/mock_streams/*.npy"))

In [ ]:
df = pd.DataFrame(np.load(mock_streams[1]), columns = ["μ_δ", "μ_α", "δ", "α", "b-r", "g", "ϕ", "λ", "μ_ϕcosλ", "μ_λ", 'stream'])
df['α'] = df['α'].apply(lambda x: x if x > 100 else x + 360)
df['stream'] = df['stream']/100
df['stream'] = df['stream'].astype(bool)
df = df[["ϕ", "λ", "μ_ϕcosλ", "μ_λ", 'b-r', 'g', 'stream']]
df

In [ ]:
selected_points = create_subset(df)

plt.figure(dpi=200, figsize=(3,3))
plt.scatter(df[~df.stream]['ϕ'], df[~df.stream]['λ'], color='grey', alpha=0.5, marker=".");
plt.scatter(selected_points['ϕ'], selected_points['λ'], color='crimson', alpha=0.5, marker=".");
plt.scatter(df[df.stream]['ϕ'], df[df.stream]['λ'], color='black', alpha=0.5, marker=".");
plt.xlabel(r"$\phi$ [$^\circ$]")
plt.ylabel(r"$\lambda$ [$^\circ$]");

In [ ]:
selected_points.stream.value_counts()

## Turn many streams into graphs

In [ ]:
"""
def create_mock_stream_graph(file, k=0):
    df = pd.DataFrame(np.load(file), columns = ["μ_δ", "μ_α", "δ", "α", "b-r", "g", "ϕ", "λ", "μ_ϕcosλ", "μ_λ", 'stream'])
    df['α'] = df['α'].apply(lambda x: x if x > 100 else x + 360)
    df['stream'] = df['stream']/100
    df['stream'] = df['stream'].astype(bool)
    
    ### create subset of stars
    df = create_subset(df)

    ### create node features and node labels
    x = torch.tensor(df[["ϕ", "λ", "μ_ϕcosλ", "μ_λ", 'b-r', 'g']].values, dtype=torch.float)
    y = torch.tensor(df[['stream']].values, dtype=torch.long).squeeze()

    ### no edges 
    # edge_index = torch.empty((2, 0), dtype=torch.long) #torch.tensor([[0, 1], [1, 2]], dtype=torch.long)
    
    ### create edges 
    # from torch_cluster import knn_graph # not working; use sklearn for now
    # edge_index = knn_graph(x, k=k, loop=False)

    ### Use scikit-learn to find k-nearest neighbors
    from sklearn.neighbors import NearestNeighbors
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(x)
    distances, indices = nbrs.kneighbors(x)
    
    ### Create edge_index tensor
    row_indices = np.repeat(np.arange(len(x)), k)
    col_indices = indices[:, 1:].flatten()  # Skip the first column (self-loops)
    edge_index = torch.tensor([row_indices, col_indices], dtype=torch.long)

    return Data(x=x, edge_index=edge_index, y=y)
"""

In [ ]:
from sklearn.neighbors import NearestNeighbors
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from sklearn.neighbors import NearestNeighbors

def create_mock_stream_graph(file, k=0):
    df = pd.DataFrame(np.load(file), columns = ["μ_δ", "μ_α", "δ", "α", "b-r", "g", "ϕ", "λ", "μ_ϕcosλ", "μ_λ", 'stream'])
    df['α'] = df['α'].apply(lambda x: x if x > 100 else x + 360)
    df['stream'] = df['stream'] / 100
    df['stream'] = df['stream'].astype(bool)
    
    ### Create subset of stars
    df = create_subset(df)

    ### Create node features and node labels
    x = torch.tensor(df[["ϕ", "λ", "μ_ϕcosλ", "μ_λ", 'b-r', 'g']].values, dtype=torch.float)
    y = torch.tensor(df[['stream']].values, dtype=torch.long).squeeze()

    ### Use scikit-learn to find k-nearest neighbors
    neigh = NearestNeighbors(n_neighbors=k+1, algorithm='auto')
    neigh.fit(x)
    knn = neigh.kneighbors(x, return_distance=False)

    ### Create edges (explicit loop-based method)
    start_edges = []
    end_edges = []
    
    for i, neighbors in enumerate(knn):
        for neighbor in neighbors[1:]:  # Skip self-loop
            start_edges.append(i)
            end_edges.append(neighbor)
            start_edges.append(neighbor)  # Add reciprocal edge
            end_edges.append(i)

    edge_index = torch.tensor([np.array(start_edges), np.array(end_edges)], dtype=torch.long)

    return Data(x=x, edge_index=edge_index, y=y)

# Example usage:
graph = create_mock_stream_graph(mock_streams[0], k=1)
print(f"Graph has {graph.num_nodes} nodes and {graph.num_edges} edges.")


In [ ]:
# plt.figure() 
# # nx.draw(torch_geometric.utils.to_networkx(graph))
# nx.draw(torch_geometric.utils.to_networkx(graph), 
#         cmap='plasma', 
#         node_color = np.arange(graph.num_nodes),
#         with_labels=True,
#         font_weight='bold',
#         font_color='white',
#         node_size=400, linewidths=6)

In [ ]:
k = 2 # set number of k-NN neighbors for the edge connectivity
train_streams = [create_mock_stream_graph(file, k=k) for file in tqdm(mock_streams[:-20], desc="Train set")]
test_streams = [create_mock_stream_graph(file, k=k) for file in tqdm(mock_streams[-20:], desc="Test set")]

# Model architecture

In [ ]:
### Uses GCNConv layers, i.e. graph convolutional layers 

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x  # Return raw logits
#return F.log_softmax(x, dim=1) # converts into log-probabilities for each class (i.e. not-stream vs. stream)

# Train 

In [ ]:
### Check for available GPUs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

### Create a DataLoader
train_loader = DataLoader(train_streams, batch_size=32, shuffle=False)

### Initialize the model, optimizer, and loss function
model = GCN(num_node_features=6, hidden_channels=64, num_classes=2)
model = model.to(device) # move onto the GPU
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
liveloss = PlotLosses(figsize=(6, 3)) 

### Training loop
model.train()
for epoch in range(100):
    logs = {}
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        
        ### Move the batch onto the GPU
        batch = batch.to(device)
        
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    loss_per_batch = total_loss / len(train_loader)
    logs['train_loss'] = loss_per_batch
    
    liveloss.update(logs)
    
    liveloss.send()
    print(f'Epoch {epoch}, Loss: {loss.item()}')

# Evaluate on test streams

In [ ]:
make_plots = True 

test_loader = DataLoader(test_streams, batch_size=1, shuffle=False) # one stream at a time
model.eval()
purities = []

for batch in test_loader:
    ### Move the batch onto the GPU
    batch = batch.to(device)
    
    # print(batch)
    df = pd.DataFrame(batch.x.cpu().detach().numpy(), columns=["ϕ", "λ", "μ_ϕcosλ", "μ_λ", 'b-r', 'g'])
    df["stream"] = batch.y.cpu().detach().numpy().astype("bool")
    out = model(batch.x, batch.edge_index)
    df["nn_score"] = out.softmax(dim=1)[:,1].cpu().detach().numpy() # probability that the star is part of a stream

    if make_plots: 
        fig, axs = plt.subplots(dpi=150, ncols=3, figsize=(9,3), tight_layout=True)
        axs[0].hist(df.nn_score, bins=25);
        axs[0].set_yscale("log")
        axs[0].set_title("NN Scores");
    
        axs[1].scatter(df["ϕ"], df["λ"], c=df.nn_score, cmap="inferno", marker=".");
        axs[1].set_title("All star scores");
        axs[1].set_xlabel(r"$\phi$ [$^\circ$]")
        axs[1].set_ylabel(r"$\lambda$ [$^\circ$]");

    top_stars = df.sort_values('nn_score',ascending=False)[:100]
    try:
        n_perfect_matches = top_stars.stream.value_counts()[True]
        purity = n_perfect_matches/len(top_stars)*100
    except:
        purity = 0
    purities.append(purity)
    
    if make_plots: 
        axs[2].scatter(top_stars[top_stars.stream]["ϕ"], top_stars[top_stars.stream]["λ"], color="crimson", marker=".");
        axs[2].scatter(top_stars[~top_stars.stream]["ϕ"], top_stars[~top_stars.stream]["λ"], color="lightpink", marker=".");
        axs[2].set_xlabel(r"$\phi$ [$^\circ$]")
        axs[2].set_ylabel(r"$\lambda$ [$^\circ$]");
        axs[2].set_title(f"Top 100 Stars (Purity = {purity:.0f}%)");

    plt.show()
    plt.close()

plt.figure(dpi=200)
plt.hist(purities, bins=20);
plt.xlabel("Purity [%]");
plt.title("Test Set Streams");
print(f"Median purity across all {len(test_loader)} test streams is {np.median(purities):.0f}%.")

# Misc.

In [ ]:
# ### kill off 99.9% of the background stars for now 
# df_bkg = df[~df.stream].sample(frac=0.001)
# df_sig = df[df.stream]
# df = pd.concat([df_bkg, df_sig]).sample(frac=1) # shuffle rows

### create a rectangular bounding box 
# def produce_rectangle(df, plot=False):
#     stream = df[df.stream]
#     delta_ϕ  = (max(stream['ϕ']) - min(stream['ϕ']))/2
#     center_ϕ = (max(stream['ϕ']) + min(stream['ϕ']))/2
#     delta_λ  = (max(stream['λ']) - min(stream['λ']))/2
#     center_λ = (max(stream['λ']) + min(stream['λ']))/2

#     box = df[(df["ϕ"] > center_ϕ - delta_ϕ) & (df["ϕ"] < center_ϕ + delta_ϕ)
#            & (df["λ"] > center_λ - delta_λ) & (df["λ"] < center_λ + delta_λ)]
    
#     box.reset_index(inplace=True)
    
#     if plot:
#         plt.scatter(df['ϕ'], df['λ'], color='grey', label='all data')
#         plt.scatter(box['ϕ'], box['λ'], color='red', label='bounding box')
#         plt.scatter(stream['ϕ'], stream['λ'], color='black', label='stream')
#         plt.legend()    
        
#     return box

# box = produce_rectangle(df, plot=True)

In [ ]:
import numpy as np
import pandas as pd

# List to store purity scores per run
purity_scores = []
all_stream_purities = []

# Run 10 separate models
for run in range(25):
    print(f"\nRunning Model {run + 1}")

    # Initialize the model
    model = GCN(num_node_features=6, hidden_channels=64, num_classes=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    # Train the model
    model.train()
    for epoch in range(100):  # Adjust epochs as needed
        total_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index)
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        #print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

    # Evaluate on the test set focusing on purity
    model.eval()
    purities = []
    stream_purities = []

    for idx, batch in enumerate(test_loader):
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index)
        preds = out.argmax(dim=1)

        # Calculate purity: TP / (TP + FP) for each stream
        true_positive = ((preds == 1) & (batch.y == 1)).sum().item()
        predicted_positive = (preds == 1).sum().item()
        purity = true_positive / predicted_positive if predicted_positive > 0 else 0
        purities.append(purity)
        stream_purities.append({'Run': run + 1, 'Stream': idx + 1, 'Purity': purity})

    avg_purity = np.mean(purities)
    purity_scores.append(avg_purity)
    all_stream_purities.extend(stream_purities)

    print(f"Model {run + 1} Average Purity: {avg_purity:.4f}")

import os
# Calculate and print the average purity over all 10 runs
overall_avg_purity = np.mean(purity_scores)
print(f"\nOverall Average Purity over 10 models: {overall_avg_purity:.4f}")

# Create DataFrame for stream purities
stream_purity_df = pd.DataFrame(all_stream_purities)

# Define the file path
csv_file = 'stream_purities_per_run_25_models.csv'

# Check if the file exists before creating it
if not os.path.exists(csv_file):
    stream_purity_df.to_csv(csv_file, index=False)
    print(f"\nPer-stream purity data saved to '{csv_file}'")
else:
    print(f"\nFile '{csv_file}' already exists. No new file was created.")

